In [1]:
%reload_ext autoreload
%autoreload 2

import json
import matplotlib.pyplot as plt
import seaborn as sns

from crowd_certain.utilities import dataset_loader, utils
from crowd_certain.utilities.settings import Settings
from crowd_certain.utilities.params import DatasetNames
from crowd_certain.utilities.utils import AIM1_3, Aim1_3_Data_Analysis_Results, AIM1_3_Plot

# Configure seaborn theme
sns.set_theme(font_scale=1.1, palette='colorblind', style='darkgrid', context='paper')

%reload_ext crowd_certain.utilities.dataset_loader
%reload_ext crowd_certain.utilities.utils
%reload_ext crowd_certain.utilities.params

In [5]:
!top -u mohammadsmajdi

=top - 15:12:49 up 6 days,  2:46,  0 users,  load average: 3.97, 1.54, 0.60
Tasks: 363 total,   6 running, 357 sleeping,   0 stopped,   0 zombie
%Cpu(s): 16.2 us,  2.0 sy,  0.0 ni, 81.8 id,  0.0 wa,  0.0 hi,  0.0 si,  0.0 st
KiB Mem : 26343524+total, 23084059+free,  5062132 used, 27532532 buff/cache
KiB Swap:        0 total,        0 free,        0 used. 25725833+avail Mem 

  PID USER      PR  NI    VIRT    RES    SHR S  %CPU %MEM     TIME+ COMMAND     
 4720 mohamma+  20   0   32.1g   1.1g 109236 R 100.0  0.5   0:09.17 python      
 4721 mohamma+  20   0   32.1g   1.1g 109016 R 100.0  0.5   0:09.19 python      
 4722 mohamma+  20   0   32.1g   1.1g 109032 R 100.0  0.5   0:09.15 python      
 4718 mohamma+  20   0   32.1g   1.1g 109024 R 100.0  0.5   0:09.18 python      
 4719 mohamma+  20   0   32.1g   1.1g 108956 R 100.0  0.5   0:09.19 python      
 3763 mohamma+  20   0    9720   1580   1224 S   0.0  0.0   0:00.01 slurm_scri+ 
 3788 mohamma+  20   0    9716   1608   1212 S   0.0  

# <span style="color:orange; font-family:PT Sans Narrow; font-size:1.31em"> 1 Loading the Data </span>

In [2]:
# Load the config.json file
config_path = '../config.json'  # Adjust this path if needed
with open(config_path, 'r') as f:
    config_dict = json.load(f)

# Create a Settings object from the config file
config = Settings(**config_dict)

data, feature_columns = dataset_loader.load_dataset(dataset_name=DatasetNames.BANKNOTE, config=config)

data['train']

------- path_all_datasets/Users/artinmajdi/Documents/GitHubs/PhD/code/submodules/crowd/crowd_certain/datasets
Loading dataset: banknote (ID: 267) from UCI ML Repository
Checking the following paths for dataset banknote:
  - /Users/artinmajdi/Documents/GitHubs/PhD/code/submodules/crowd/crowd_certain/datasets/banknote/banknote.data (exists: False)
  - /Users/artinmajdi/Documents/GitHubs/PhD/code/submodules/crowd/crowd_certain/datasets/banknote/banknote.csv (exists: True)
Found dataset in local cache at /Users/artinmajdi/Documents/GitHubs/PhD/code/submodules/crowd/crowd_certain/datasets/banknote/banknote.csv


,variance,skewness,curtosis,entropy,true
0,3.62160,8.6661,-2.80730,-0.44699,0
2,3.86600,-2.6383,1.92420,0.10645,0
3,3.45660,9.5228,-4.01120,-3.59440,0
4,0.32924,-4.4552,4.57180,-0.98880,0
5,4.36840,9.6718,-3.96060,-3.16250,0
...,...,...,...,...,...
1365,-4.50460,-5.8126,10.88670,-0.52846,1
1366,-2.41000,3.7433,-0.40215,-1.29530,1
1367,0.40614,1.3492,-1.45010,-0.55949,1
1368,-1.38870,-4.8773,6.47740,0.34179,1


# <span style="color:orange; font-family:PT Sans Narrow; font-size:1.3em"> 1. More detailed version. Repeating the experiments for only 20 workers => to measure confidence score </span>

## <span style="font-family:PT Sans Narrow; font-size:1.3em"> 1.1 Measuring prob/uncertainties </span>

In [ ]:
ARLS = {'num_labelers': 10,
        'low_dis':      0.3,
        'high_dis':     0.9}

predicted_labels, uncertainty, true_labels, labelers_strength = funcs.apply_technique_aim_1_3( data = data,
                                                                                               ARLS = ARLS,
                                                                                               num_simulations = 20,
                                                                                               feature_columns = feature_columns)

labels_all_workers         = predicted_labels['test']['mv']
uncertainty_all_workers    = uncertainty['test']
truth                      = true_labels['test'].truth

uncertainty['test'].head(3).append(labelers_strength.T).round(decimals=3)

## <span style="font-family:PT Sans Narrow; font-size:1.3em"> 1.2 Measuring weights for each labeler </span>

***
<span style="color:grey; font-family:PT Sans narrow; font-size:1.3em"> 1.2.1 First Method: </span>

$ T_{x,a,j} = 1 - u_{j} $

***
<span style="color:grey; font-family:PT Sans narrow; font-size:1.3em"> 1.2.2 Second Method: </span>

$ T_{x,a,j} = \begin{array}{cc} 1 - u_{j} & y_{a,j} = y'_{j}  \\ 0 & y_{a,j} \neq y'_{j} \end{array} $

***

<span style="color:grey; font-family:PT Sans narrow; font-size:1.3em"> 1.2.3 Measuring average weight </span>

$ \hat{w}_{a,j} = \frac {1}{N} \sum_{x} T_{x,a,j}$

$ w_{a,j} = \frac {\hat{w}_{a,j}} {\sum_{a=1}^{L} \hat{w}_{a,j}} $

***
## <span style="font-family:PT Sans Narrow; font-size:1.3em"> 1.3 Weighted majority voting </span>


<span style="color:grey; font-family:PT Sans narrow; font-size:1.3em"> 1.3.1 Applying the weights to predicted probabilities </span>

$ \hat{p}^{prob}_{j} = \sum_{a=1}^{L} p_{a,j} * w_{a,j} $



<span style="color:grey; font-family:PT Sans narrow; font-size:1.3em"> 1.3.2 Applying the weights to predicted labels </span>

$ \hat{p}^{binary}_{j} = \sum_{a=1}^{L} y_{a,j} * w_{a,j}$ where $y_{a,j} = (p_{a,j} > 0.5) $
***

In [ ]:
weights, prob_weighted = funcs.aim1_3_measuring_weights( labels_all_workers=labels_all_workers, uncertainty_all_workers=uncertainty_all_workers)

### <span style="color:grey; font-family:PT Sans narrow; font-size:1.3em"> 1.3.3 Measuring the weighted MV using only the measured weights (without confidence scores) </span>

In [ ]:
# measuring the new accuracies
acc2 = ( (prob_weighted > 0.5).T == truth ).mean(axis=1)
acc2['num_labelers'] = ARLS['num_labelers']

accuracy2 = pd.DataFrame( {'accuracy': acc2}).T.set_index('num_labelers')

accuracy2

In [ ]:
F, acc = funcs.aim1_3_measure_confidense_score(delta=labels_all_workers, weights=weights, conf_score_strategy=1, num_labelers=ARLS['num_labelers'], truth=true_labels['test'].truth)

acc

In [ ]:
# acc2 = ((F1.method1>0.5) == (truth > 0.5) ).mean(axis=0)
# acc2
# truth
# F1.method1>0.5
F1

# <span style="color:orange; font-family:PT Sans Narrow; font-size:1.3em"> 2. Benchmark </span>

## <span style="font-family:PT Sans Narrow; font-size:1.3em"> 2.1 Overall quality of different workers </span>


Estimating the overall qualities of different workers is not a new research topic in the crowdsourcing learning community. To the best of the authors’ knowledge, there exist many state-of-the-art algorithms, such as Dawid–Skene [1], ZenCrowd , KOS [9], and DEW [15, 23]. However, none of them exploit feature vectors of instances, which makes it impossible to take full advantage of the statistical characteristics of the available data when evaluating the label qualities. According to the observation by [30], in traditional supervised learning, there exists a schema to exhibit the relationship between data features and the ground-truth labels. For example, suppose there exists a high-quality worker; the data schema will be well-inherited in their labels, because the difference between their labels and ground-truth labels is small. Meanwhile, suppose there exists a low-quality worker, the data schema may be broken because their labels will be very different from the ground-truth labels. Therefore, we can estimate the overall quality of a worker by evaluating how well the schema is inherited in their labels. Specifically, we can first extract all training instances’ feature vectors and the corresponding crowd labels provided by the jth worker to form a new single-label data set. Then, we use tenfold cross-validation to evaluate the classification accuracy of a classifier. In theory, this classifier can be any classifier. Finally, we define the overall quality of the jth worker as the classification accuracy of the built classifier. The detailed formula can be expressed as


$ \tau_{a} = \frac {\sum_{i=1}^{n} \delta \Big( f_{a}(x_{i}) , I_{i,a}  \Big)}{n} $

where n is the size of the extracted data set and $f_{j}(x_{i})$ is the class label of the feature vector $x_{i}$ predicted by the built classifier.

***

## <span style="font-family:PT Sans Narrow; font-size:1.3em"> 2.2 Specific quality of the $j_{th}$ worker for the $i_{th}$ instance ($s_{ij}$) </span>

$ s_{x,a} = \sum^{a'=L}_{ (a'=1) \land (a' \neq a) } \delta \Big( l_{x,a},l_{x,a'} \Big) $

***
$ \gamma_{x,a} =\tau_{x,a}(1 + s_{x,a}^{2}) $

***
$ w'_{x,a} = \frac {1} {1 + e^{-\gamma_{x,a}} } $

***
$ Z = \frac {1}{L} \sum_{a=1}^{L}w'_{x,a}  $

‍‍‍``` Z is a normalization constant, which ensures that the sum of all crowd label weights for the ith instance is still equal to m ```

***
$ w_{x,j} = \frac {1}{Z} w'_{x,j}  $

In [ ]:
labels = funcs.measuring_Tao_Weights(delta=predicted_labels['test']['simulation_0'] , true_labels=true_labels['test'].drop(columns=['truth']))

# measuring accuracy
accuracy2['WMV_Tao'] = ( labels['WMV_Tao'] == true_labels['test'].truth ).mean(axis=0)
accuracy2['MV']      = ( labels['MV']      == true_labels['test'].truth ).mean(axis=0)

accuracy2

# <span style="color:orange; font-family:PT Sans Narrow; font-size:1.3em"> 3. Confidense score: _Weighted-soft-MV_ </span>

## <span style="font-family:PT Sans Narrow; font-size:1.3em"> 3.1 Measuring the certainty score of majority class $ P_{x,j} $ </span>


In actual formula this is divided by weights.sum(axis=1). But because weights sum to 1, its values would be 1.

Also pandas automatically transfers the binary values in delta\[disease\] to float before doing the multiplication.

where $\delta(y_{a,j},+)$ is $1$ if $y_{a,j}$ is positive (TRUE) otherwise $0$. $\delta(y_{a,j},-)$ is $1$ if $y_{a,j}$ is negative (FALSE) otherwise $0$

$ P_{x,j} = \frac { \sum_{a=1}^{L} {ω_{a,j} δ(y_{a,j},+)} } { \sum_{a=1}^{L} {ω_{a,j} δ(y_{a,j},+)}  +  \sum_{a=1}^{L} {ω_{a,j} δ(y_{a,j},-)} }$

***
## <span style="font-family:PT Sans Narrow; font-size:1.3em"> 3.2 Certainty of majority class for both positive & negative labels </span>

$F_{x,j} = max \Big(P_{x,j} , 1-P_{x,j} \Big)$

In [ ]:
F1 = funcs.aim1_3_measure_confidense_score(delta=labels_all_workers, weights=weights, conf_score_strategy=1, num_labelers=ARLS['num_labelers'], truth=true_labels['test'].truth)
F1

***
# <span style="color:orange; font-family:PT Sans Narrow; font-size:1.3em"> 4. Confidense score: _Beta-soft-MV_ </span>

> Note: _This is measured only for METHOD1 since it has a higher accuracy_

## <span style="font-family:PT Sans Narrow; font-size:1.3em"> 4.1 Measuring the certainty score of majority class  $f_{x,j}^{-+}$ </span>



$f^{+}_{x,j}≔1+\sum_{a=1}^{L}ω_{a,j}  \delta \big( y_{a,j},+ \big) $

$f_{x,j}^{-}≔1+\sum_{a=1}^{L}ω_{a,j}  \delta \big( y_{a,j},- \big) $

***
## <span style="font-family:PT Sans Narrow; font-size:1.3em"> 4.2 Measuring the regularized incomplete beta function </span>


$I_{x} (α,β)=F(x;α,β)=\frac{ B(x;α,β) }{B(α,β)} $

$ bdtrc(k,n,p) = I_{p} \Big( \lfloor {k} \rfloor + 1 , n - \lfloor {k} \rfloor \Big) = \sum_{j = \lfloor {k} \rfloor + 1} ^ {n} \binom {n}{j}p^{j}(1-p)^{n-j} $

> [source](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.bdtrc.html)


## <span style="font-family:PT Sans Narrow; font-size:1.3em"> 4.3 Certainty of majority class for both positive & negative labels </span>

$F_{x,j} = max(I_{p} , 1-I_{p})$

In [ ]:
F2 = funcs.aim1_3_measure_confidense_score(delta=labels_all_workers, weights=weights, method=2, num_labelers=ARLS['num_labelers'], truth=true_labels['test'].truth)
F2